# Understanding the Impact of Image Quality and Distance of Objects to Object Detection Performance

***A reproduction*** 

## Downscale Data

Folder expansion: Some datasets come with multiple subfolders not needed for the purposes of our reproduction study. Example:

```
img
|_dir1 
    |_img1.png
    |_img2.png
    |_img3.png
|_dir2
    |_img1.png
    |_img2.png
    |_img3.png
|_dir3
(...)
```

In [ ]:
from data_processing.expand_folders import expand_folders
expand_folders("../datasets/ECP2dot5D_day_labels_val/ECP2dot5D/day/labels/val")

### Spatial and Amplitudinal Resolution Downsampling


Spatial - 1.42x Downsampling

In [ ]:
from data_processing.expand_dataset import expand_dataset

INPUT_IMAGE_DIR = "../datasets/ECP/day/img/val"
INPUT_LABEL_DIR = "../datasets/ECP2dot5D_day_labels_val/ECP2dot5D/day/labels/val"
DATASET_OUTPUT_DIR = "../datasets/spatially_compressed"
label_values_to_scale = ["imageheight", "imagewidth", "x0", "y0", "x1", "y1"]
OUTPUT_IMG_DIR = f"{DATASET_OUTPUT_DIR}/img"
OUTPUT_LABEL_DIR = f"{DATASET_OUTPUT_DIR}/labels"

expand_dataset(
    input_dir=INPUT_IMAGE_DIR,
    label_dir=INPUT_LABEL_DIR,
    label_values_to_scale=label_values_to_scale,
    output_img_dir=OUTPUT_IMG_DIR,
    output_label_dir=OUTPUT_LABEL_DIR,
    scale_factors=[720.0/1024], #value from paper 
    qp_values=[],
)

Amplitudinal Downsampling

In [1]:
from data_processing.expand_dataset import expand_dataset

INPUT_IMAGE_DIR = "../datasets/ECP/day/img/val"
INPUT_LABEL_DIR = "../datasets/ECP2dot5D_day_labels_val/ECP2dot5D/day/labels/val"
DATASET_OUTPUT_DIR = "../datasets/eurocity_original_amplitudinally_compressed"
label_values_to_scale = ["imageheight", "imagewidth", "x0", "y0", "x1", "y1"]
OUTPUT_IMG_DIR = f"{DATASET_OUTPUT_DIR}/img"
OUTPUT_LABEL_DIR = f"{DATASET_OUTPUT_DIR}/labels"
COMPRESSION_METADATA_DIR = f"{DATASET_OUTPUT_DIR}/metadata"

qp_values = [16, 24, 34, 38, 46] #values from paper

expand_dataset(
    input_dir=INPUT_IMAGE_DIR,
    label_dir=INPUT_LABEL_DIR,
    label_values_to_scale=label_values_to_scale,
    scale_factors=[],
    output_img_dir=OUTPUT_IMG_DIR,
    output_label_dir=OUTPUT_LABEL_DIR,
    metadata_dir=COMPRESSION_METADATA_DIR,
    qp_values=qp_values, #values from paper
    expansion = "amplitudinal"
)

Starting Processing amplitudinal downsampling with 23 parallel processes...


Processing amplitudinal downsampling: 100%|██████████| 860/860 [01:46<00:00,  8.06it/s]

All images expanded and saved to D:\code-projects\Python_projects\datasets\eurocity_original_amplitudinally_compressed\img and D:\code-projects\Python_projects\datasets\eurocity_original_amplitudinally_compressed\labels


Mixed Downsampling - Combined Set of both

In [ ]:
from data_processing.expand_dataset import expand_dataset

INPUT_IMAGE_DIR = "../datasets/ECP/day/img/val"
INPUT_LABEL_DIR = "../datasets/ECP2dot5D_day_labels_val/ECP2dot5D/day/labels/val"
DATASET_OUTPUT_DIR = "../datasets/test"
label_values_to_scale = ["imageheight", "imagewidth", "x0", "y0", "x1", "y1"]
OUTPUT_IMG_DIR = f"{DATASET_OUTPUT_DIR}/img"
OUTPUT_LABEL_DIR = f"{DATASET_OUTPUT_DIR}/labels"
COMPRESSION_METADATA_DIR = f"{DATASET_OUTPUT_DIR}/metadata"
qp_values = [16, 24, 34, 38, 46] #values from paper

expand_dataset(
    input_dir=INPUT_IMAGE_DIR,
    label_dir=INPUT_LABEL_DIR,
    label_values_to_scale=label_values_to_scale,
    output_img_dir=OUTPUT_IMG_DIR,
    output_label_dir=OUTPUT_LABEL_DIR,
    metadata_dir=COMPRESSION_METADATA_DIR,
    scale_factors=[1, 720.0/1024, 854.0/ 1920], #values from paper
    qp_values=[0, 5, 10, 16, 20, 24, 28, 34, 38, 42, 46, 51], #values from paper
    expansion="mixed", 
    subsample_spatial=True,
    subsample_amplitudinal=True,
)

### Convert to YOLO format

In [ ]:
from datasets.to_yolo_format import to_yolo_format

labels_dir = "../datasets/mixed/labels"     # YOLO output
images_dir = "../datasets/mixed/img"     # Images directory
dataset_root = "../datasets/mixed"          # Root directory
split = 0.8


to_yolo_format(
    labels_dir=labels_dir,
    images_dir=images_dir,
    dataset_root=dataset_root,
    split=split,
)

## Train/Load Model

#### Train

In [ ]:
import sys
import os
import torch
import argparse
from pathlib import Path
yolov5_dir = os.path.abspath('yolov5')
if yolov5_dir not in sys.path:
    sys.path.append(yolov5_dir)
from yolov5.train import main, Callbacks
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cfg = "ra_yolo5l.yaml"  
opt = argparse.Namespace(
    weights='yolov5l.pt',  # Model weights
    cfg=cfg,  # Empty to use weights' default config
    data=os.path.abspath('../datasets/mixed/data.yaml'),  # Absolute path to dataset
    hyp=os.path.join(yolov5_dir, 'data/hyps/hyp.scratch-low.yaml'),  # Hyperparameters
    epochs=1,
    batch_size=1,
    imgsz=800,  # Fixed from invalid imgsz=4
    rect=False,
    resume=False,
    nosave=False,
    noval=False,
    noautoanchor=False,
    noplots=False,
    evolve=None,
    evolve_population=os.path.join(yolov5_dir, 'data/hyps'),
    resume_evolve=None,
    bucket='',
    cache=None,
    image_weights=False,
    device=device,
    multi_scale=False,
    single_cls=False,
    optimizer='SGD',
    sync_bn=False,
    workers=8,
    project=os.path.join(yolov5_dir, 'runs/train'),
    name='exp',
    exist_ok=True,
    quad=False,
    cos_lr=False,
    label_smoothing=0.0,
    patience=100,
    freeze=[0],  # Freeze first 10 layers
    save_period=-1,
    seed=0,
    local_rank=-1,
    ra_yolo=True,
    entity=None,
    upload_dataset=False,
    bbox_interval=-1,
    artifact_alias='latest',
    ndjson_console=False,
    ndjson_file=False
)

main(opt, callbacks=Callbacks())

#### Extract results

In [ ]:
import torch 
from yolov5.val import run  as yolov5_run_val

data = '../datasets/mixed/data.yaml'  # Path to your dataset YAML file
weights = 'yolov5/runs/train/exp/weights/best.pt'      # Path to your model weights file
batch_size = 1             # Batch size for validation           # Image size for inference
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # Automatically select device
confidence_threshold = 0.1


# Run the validation
results, maps, times = yolov5_run_val(
    data=data,              # Dataset configuration
    imgsz=4,
    weights=weights,        # Model weights
    batch_size=batch_size,  # Batch size         # Image size
    device=device,          # Device to run on
    task='val',             # Task type: validation
    save_txt=True,         # Don’t save results to text files
    save_json=True,        # Don’t save results to JSON
    plots=True,             # Generate plots (saved to runs/val/)
    confidence=confidence_threshold,  # Confidence threshold for predictions
)

# Extract metrics from r
mp, mr, map50, map, box_loss, obj_loss, cls_loss = results  # Mean Precision, Mean Recall, mAP@0.5, mAP@0.5:0.95
print(f'Mean Precision: {mp:.4f}')
print(f'Mean Recall: {mr:.4f}')
print(f'mAP@0.5: {map50:.4f}')
print(f'mAP@0.5:0.95: {map:.4f}')

In [ ]:
def extract_predictions_from_run(json_path, label_dir=None):
    import json
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    predictions = {}
    labels = {}
    
    
    for item in data:
        if item['image_id'] not in predictions.keys():
            predictions[item['image_id']] = []
        if item['image_id'] not in labels.keys():
            labels[item['image_id']] = json.load(open(f"{label_dir}/{item['image_id']}.json")) if label_dir else {}
            
        predictions[item['image_id']].append(item)
    
    return predictions, labels

predictions, labels = extract_predictions_from_run(json_path="yolov5/runs/val/exp13/best_predictions.json", label_dir="../datasets/mixed/labels")

In [ ]:
import pandas as pd
from tqdm import tqdm

def calculate_iou(bbox1, bbox2):
    # Existing IoU calculation remains unchanged
    x0_pred, y0_pred, x1_pred, y1_pred = bbox1
    x0_gt, y0_gt, x1_gt, y1_gt = bbox2
    
    x0_inter = max(x0_pred, x0_gt)
    y0_inter = max(y0_pred, y0_gt)
    x1_inter = min(x1_pred, x1_gt)
    y1_inter = min(y1_pred, y1_gt)
    
    if x0_inter < x1_inter and y0_inter < y1_inter:
        intersection_area = (x1_inter - x0_inter) * (y1_inter - y0_inter)
    else:
        intersection_area = 0
    
    area_pred = (x1_pred - x0_pred) * (y1_pred - y0_pred)
    area_gt = (x1_gt - x0_gt) * (y1_gt - y0_gt)
    union_area = area_pred + area_gt - intersection_area
    
    return intersection_area / union_area if union_area != 0 else 0.0

def calculate_distance_to_gt(label):
    xyz = label[4]
    x, y, z = xyz["x"], xyz["y"], xyz["z"]

    return (x**2 + y**2 + z**2)**0.5 if x is not None and y is not None and z is not None else 0.0

def create_predictions_dataframe(predictions, labels):
    rows = []  # Collect all rows here for batch DataFrame creation
    
    for im_name in tqdm(predictions.keys()):
        # Extract resolutions from filename
        parts = im_name.split('_')
        spatial_res = float(parts[5][:2])
        amp_val = parts[6][2:].split('.')[0]  # Handle file extensions
        amplitudal_res = int(amp_val) if amp_val != '' else 0
        
        # Get current predictions and labels
        curr_preds = predictions[im_name]
        curr_labels = [obj for obj in labels[im_name]["children"] 
                      if obj['identity'] == "pedestrian"]
        
        gt_bboxes = [
            [obj['x0'], obj['y0'], obj['x1'], obj['y1'], obj['3dp'] if '3dp' in obj else {"x": None, "y": None, "z": None}]
            for obj in curr_labels
        ]
        pred_bboxes = [pred['bbox'] for pred in curr_preds]
        
        matched_pred_indices = set()  # Track indices of matched predictions
        
        # Process ground truths
        for gt_bbox in gt_bboxes:
            matched = False
            for i, pred_bbox in enumerate(pred_bboxes):
                if i in matched_pred_indices:
                    continue  # Skip already matched predictions
                if calculate_iou(pred_bbox, gt_bbox[:4]) > 0.5:
                    matched_pred_indices.add(i)
                    matched = True
                    break  # Stop after first match
            
            # Add row for ground truth
            rows.append({
                'image_id': im_name,
                'distance': calculate_distance_to_gt(gt_bbox),
                'spatial_res': spatial_res,
                'amplitudal_res': amplitudal_res,
                'distance_to_gt': 0,
                'tp': int(matched),
                'fp': 0,
                'fn': int(not matched),
                'label': 'gt'
            })
        
        # Process unmatched predictions (false positives)
        for i, _ in enumerate(pred_bboxes):
            if i not in matched_pred_indices:
                rows.append({
                    'image_id': im_name,
                    'distance': 0,
                    'spatial_res': spatial_res,
                    'amplitudal_res': amplitudal_res,
                    'distance_to_gt': 0,
                    'tp': 0,
                    'fp': 1,
                    'fn': 0,
                    'label': 'pred'
                })
    
    return pd.DataFrame(rows)
    
create_predictions_dataframe(predictions, labels)

## Visualizing Results

#### Reproducing figures from the paper

#### Figure 5

In [ ]:
from visualization.xy_lineplot import basic_lineplot, multi_lineplot

data_dict = {
    "EuroCity Original -> Finetuned Model": ([0, 1, 2, 3, 4, 5], [0, 1, 4, 9, 16, 25]),
    "EuroCity 1.42 -> Finetuned Model": ([0, 1, 2, 6, 8, 9], [0, 1, 5, 10, 16, 27]),
    "EuroCity Original": ([0, 2, 3, 5, 7, 9], [0, 1, 2, 3, 12, 20]), 
    "EuroCity 1.42": ([0, 2, 2, 4, 8, 9], [0, 1, 4, 9, 16, 25])
}
## Figure 5 in the paper 
multi_lineplot(data_dict, xlabel="Megabytes/Image", ylabel="map@50 - All Category")

In [ ]:
from visualization.xy_lineplot import basic_lineplot, multi_lineplot

data_dict = {
    "EuroCity Original": ([0, 1, 2, 3, 4, 5], [25, 16, 10, 7, 3, 1]),
    "EuroCity 1.42 -> Finetuned Model": ([0, 1, 2, 6, 8, 9], [24, 13, 10, 5, 1, 0]),
}
## Figure 5 in the paper 
multi_lineplot(data_dict, xlabel="Distance(m)", ylabel="Recall-Pedestrian")